In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import BeerBenchmark

benchmark = BeerBenchmark(data_dir_path="../data/")
benchmark

17:13:55 INFO:Extracting Beer...
17:13:55 INFO:Reading Beer row_dict...
17:13:55 INFO:Reading Beer train.csv...
17:13:55 INFO:Reading Beer valid.csv...
17:13:55 INFO:Reading Beer test.csv...


<BeerBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Beer/beer_exp_data.zip

## Preprocess

In [5]:
attr_list = ['Beer_Name', 'Brew_Factory_Name', 'Style', 'ABV']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/7345 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'Beer_Name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_Beer_Name': {
        'source_attr': 'Beer_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'Brew_Factory_Name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_Brew_Factory_Name': {
        'source_attr': 'Brew_Factory_Name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'Style': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_Style': {
        'source_attr': 'Style',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'ABV': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_info_dict

17:13:55 INFO:For attr=Beer_Name, computing actual max_str_len
17:13:55 INFO:For attr=Beer_Name, using actual_max_str_len=18
17:13:55 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:13:58 INFO:For attr=Brew_Factory_Name, computing actual max_str_len
17:13:58 INFO:For attr=Brew_Factory_Name, using actual_max_str_len=20
17:13:58 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:14:01 INFO:For attr=Style, computing actual max_str_len
17:14:01 INFO:actual_max_str_len=13 must be even to enable NN pooling. Updating to 14
17:14:01 INFO:For attr=Style, using actual_max_str_len=14
17:14:01 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:14:04 INFO:For attr=ABV, computing actual max_str_len
17:14:04 INFO:actual_max_str_len=7 must be even to enable NN pooling. Updating to 8
17:14:04 INFO:For attr=ABV, using actual_max_str_len=8


{'Beer_Name': NumericalizeInfo(source_attr='Beer_Name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=18, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_Beer_Name': NumericalizeInfo(source_attr='Beer_Name', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=32,
    eval_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    row_numericalizer,
    ann_k=ann_k,
    eval_with_clusters=False
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.5',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
17:14:05 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:14:05 INFO:TPU available: None, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule)

17:14:05 INFO:Train pair count: 40
17:14:05 INFO:Valid pair count: 14
17:14:05 INFO:Test pair count: 14
17:14:05 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 10.5 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
7.4 M     Trainable params
3.1 M     Non-trainable params
10.5 M    Total params
41.927    Total estimated model params size (MB)
17:14:37 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 10.5 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
7.4 M     Trainable params
3.1 M     Non-trainable params
10.5 M    Total params
41.927    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [15]:
model.blocker_net.get_signature_weights()

{'Beer_Name': 0.1647830456495285,
 'semantic_Beer_Name': 0.16125541925430298,
 'Brew_Factory_Name': 0.15044645965099335,
 'semantic_Brew_Factory_Name': 0.1581915020942688,
 'Style': 0.12117814272642136,
 'semantic_Style': 0.1259252429008484,
 'ABV': 0.11822022497653961}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

17:16:36 INFO:Validating best model...


{'valid_f1_at_0.3': 0.8750000000000001,
 'valid_f1_at_0.5': 1.0,
 'valid_f1_at_0.7': 0.782608695652174,
 'valid_f1_at_0.9': 0.35294117647058826,
 'valid_pair_entity_ratio_at_0.3': 0.6428571428571429,
 'valid_pair_entity_ratio_at_0.5': 0.5,
 'valid_pair_entity_ratio_at_0.7': 0.32142857142857145,
 'valid_pair_entity_ratio_at_0.9': 0.10714285714285714,
 'valid_precision_at_0.3': 0.7777777777777778,
 'valid_precision_at_0.5': 1.0,
 'valid_precision_at_0.7': 1.0,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 1.0,
 'valid_recall_at_0.7': 0.6428571428571429,
 'valid_recall_at_0.9': 0.21428571428571427}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

17:16:37 INFO:Train pair count: 40
17:16:37 INFO:Valid pair count: 14
17:16:37 INFO:Test pair count: 14
17:16:37 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.9032258064516129,
  'test_f1_at_0.5': 1.0,
  'test_f1_at_0.7': 0.923076923076923,
  'test_f1_at_0.9': 0.4444444444444445,
  'test_pair_entity_ratio_at_0.3': 0.6071428571428571,
  'test_pair_entity_ratio_at_0.5': 0.5,
  'test_pair_entity_ratio_at_0.7': 0.42857142857142855,
  'test_pair_entity_ratio_at_0.9': 0.14285714285714285,
  'test_precision_at_0.3': 0.8235294117647058,
  'test_precision_at_0.5': 1.0,
  'test_precision_at_0.7': 1.0,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 1.0,
  'test_recall_at_0.7': 0.8571428571428571,
  'test_recall_at_0.9': 0.2857142857142857}]